# 🧠 Multi-Label Classification — Google Colab
Proyek ini menggunakan dataset makanan untuk melakukan **multi-label classification**:
- Target 1: 🥗 `Veg_non` → klasifikasi makanan veg / non-veg
- Target 2: 👶🧑‍🎓🧓 Kategori umur → Anak / Remaja / Dewasa

Model yang digunakan: `DecisionTreeClassifier` dengan `MultiOutputClassifier`.


In [17]:
# 📌 1. Upload Dataset
# The file Recomfood.xlsx already exists, skipping upload.
# from google.colab import files
# uploaded = files.upload('Recomfood.xlsx')

In [18]:
# 📚 2. Load Dataset
import pandas as pd
df = pd.read_excel('Recomfood.xlsx')
df.head()

,User_ID,Age,Gender,Food,Category,Veg_non,Rating
0,1,14,Male,summer squash salad,Primary,veg,4
1,1,14,Male,apple rabdi,Dessert,veg,3
2,1,14,Male,southern fried chicken tenders,Snack,non-veg,5
3,1,14,Male,chocolate appe,Snack,veg,4
4,2,37,Female,chocolate samosa,Snack,veg,1


## 🧭 3. Eksplorasi Data

In [19]:
df.info()
df.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 511 entries, 0 to 510
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   User_ID   511 non-null    int64 
 1   Age       511 non-null    int64 
 2   Gender    511 non-null    object
 3   Food      511 non-null    object
 4   Category  511 non-null    object
 5   Veg_non   511 non-null    object
 6   Rating    511 non-null    int64 
dtypes: int64(3), object(4)
memory usage: 28.1+ KB


,User_ID,Age,Gender,Food,Category,Veg_non,Rating
count,511.000000,511.000000,511,511,511,511,511.000000
unique,NaN,NaN,2,45,3,2,NaN
top,NaN,NaN,Male,pesto fish kebabs,Primary,veg,NaN
freq,NaN,NaN,260,21,189,360,NaN
mean,49.068493,32.281800,NaN,NaN,NaN,NaN,5.438356
std,28.739213,14.089867,NaN,NaN,NaN,NaN,2.866236
min,1.000000,10.000000,NaN,NaN,NaN,NaN,1.000000
25%,25.000000,20.000000,NaN,NaN,NaN,NaN,3.000000
50%,49.000000,33.000000,NaN,NaN,NaN,NaN,5.000000
75%,72.000000,44.000000,NaN,NaN,NaN,NaN,8.000000


## 🧼 4. Preprocessing

In [20]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

df = df.copy()


le_veg = LabelEncoder()
df['Veg_non_encoded'] = le_veg.fit_transform(df['Veg_non'])


def categorize_age(age):
    if age <= 17:
        return 'Anak'
    elif 18 <= age <= 30:
        return 'Remaja'
    else:
        return 'Dewasa'

df['Age_category'] = df['Age'].apply(categorize_age)

le_agecat = LabelEncoder()
df['Age_category_encoded'] = le_agecat.fit_transform(df['Age_category'])

df[['Veg_non','Veg_non_encoded','Age','Age_category','Age_category_encoded']].head()

,Veg_non,Veg_non_encoded,Age,Age_category,Age_category_encoded
0,veg,1,14,Anak,0
1,veg,1,14,Anak,0
2,non-veg,0,14,Anak,0
3,veg,1,14,Anak,0
4,veg,1,37,Dewasa,1


In [21]:

feature_cols = ['User_ID', 'Age', 'Gender', 'Food', 'Category']
df_encoded = df.copy()
label_encoders = {}

for col in ['Gender','Food','Category']:
    le = LabelEncoder()
    df_encoded[col] = le.fit_transform(df_encoded[col])
    label_encoders[col] = le

X = df_encoded[feature_cols]
Y = df_encoded[['Veg_non_encoded','Age_category_encoded']]

X.head()

,User_ID,Age,Gender,Food,Category
0,1,14,1,42,1
1,1,14,1,3,0
2,1,14,1,38,2
3,1,14,1,18,2
4,2,37,0,21,2


## ✂️ 5. Split Data (Train & Test)

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape

((408, 5), (103, 5))

## 🤖 6. Model Multi-Label Classification

In [23]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.multioutput import MultiOutputClassifier

base_model = DecisionTreeClassifier(random_state=42)
multi_model = MultiOutputClassifier(base_model)

multi_model.fit(X_train, Y_train)
Y_pred = multi_model.predict(X_test)

## 📈 7. Evaluasi Model

In [24]:
from sklearn.metrics import accuracy_score, classification_report

print('🎯 Akurasi Veg_non:', accuracy_score(Y_test.iloc[:,0], Y_pred[:,0]))
print('🎯 Akurasi Age_category:', accuracy_score(Y_test.iloc[:,1], Y_pred[:,1]))

print('\n📊 Classification Report - Veg_non')
print(classification_report(Y_test.iloc[:,0], Y_pred[:,0]))

print('\n📊 Classification Report - Age Category')
print(classification_report(Y_test.iloc[:,1], Y_pred[:,1]))

🎯 Akurasi Veg_non: 1.0
🎯 Akurasi Age_category: 1.0

📊 Classification Report - Veg_non
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        34
           1       1.00      1.00      1.00        69

    accuracy                           1.00       103
   macro avg       1.00      1.00      1.00       103
weighted avg       1.00      1.00      1.00       103


📊 Classification Report - Age Category
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        26
           1       1.00      1.00      1.00        61
           2       1.00      1.00      1.00        16

    accuracy                           1.00       103
   macro avg       1.00      1.00      1.00       103
weighted avg       1.00      1.00      1.00       103

